In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

2025-08-18 17:34:15.197014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 17:34:15.931759: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-18 17:34:15.931858: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-18 17:34:15.935311: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 17:34:16.344016: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 17:34:16.347504: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

### Copying weights into the model

In [3]:
model.compile()

In [4]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Test with background data

In [5]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:]
scale_data = data_file['data']['Normalisation']['norm_scale'][:].flatten()
offset_data =  data_file['data']['Normalisation']['norm_bias'][:].flatten()

X_flat = X.reshape(X.shape[0], -1)
X_int = (X_flat * scale_data) + offset_data
X_flat_int = X_int.astype('int')
input_quantizer = quantized_bits(14, 5, alpha=1)
output_quantizer = quantized_bits(18,13,alpha=1)
bias_quantizer = quantized_bits(18,12,alpha=1)

In [6]:
X_for_qk = X_flat_int[0].astype('float')
X_biased = bias_quantizer(X_for_qk - offset_data).numpy().astype('float')
X_scaled = bias_quantizer(X_biased / scale_data)
X_scaled_quantized_2 = input_quantizer(X_scaled)

In [8]:
pred_qkeras = output_quantizer(model.predict(X_scaled_quantized_2.numpy().reshape(1,57)))

1/1 [==============================] - 1s 649ms/step


### Conversion to a QONNX model

In [9]:
import onnx
import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes

qonnx_model, external_storage = from_keras(model)
qonnx_model = ModelWrapper(qonnx_model)
qonnx_model = qonnx_model.transform(InferShapes())

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(
2025-08-18 17:39:13.401560: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-08-18 17:39:13.402198: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-08-18 17:39:13.466994: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-08-18 17:39:13.467561: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [10]:
idict = {qonnx_model.graph.input[0].name: X_scaled_quantized_2.numpy().reshape(1,57)}
odict = oxe.execute_onnx(qonnx_model, idict, True)
pred_qonnx = odict[qonnx_model.graph.output[0].name]

print("qkeras:\t",pred_qkeras)
print("qonnx:\t",pred_qonnx)


qkeras:	 tf.Tensor([[4.4375]], shape=(1, 1), dtype=float32)
qonnx:	 [[4.4136963]]
